<a href="https://colab.research.google.com/github/tada20001/NLP_2023/blob/main/CH19_08_BERTopic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bertopic[visualization]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 108.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 14.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 118.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 65.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  

### 1. BERTopic
--------------
BERTopic은 BERT embeddings과 클래스 기반(class-based) TF-IDF를 활용하여 중요한 단어를 유지하면서 쉽게 해석할 수 있는 클러스터를 만드는 토픽 모델링 방법임. 

BERTopic 알고리즘은 세가지 과정을 거침
* (step 1) 텍스트 데이터를 SBERT로 임베딩
  * "paraphrase-MiniLM-L6-v2" : 영어 데이터로 학습된 SBERT
  * "paraphrase-multilingual-MiniLM-L12-v2" : 50개 이상의 언어로 학습된 다국어 SBERT
* (step 2) 문서를 군집화
  * UMAP을 사용하여 임베딩 차원을 줄이고 HDBSCAN 기술을 사용하여 차원 축소된 임베딩을 클러스터링하고 의미적으로 유사한 문서 클러스터 생성
* (step 3) 토픽 표현 생성
  * 클래스 기반 TF-IDF로 토픽 추출

### 2. 데이터 로드
---------------


In [2]:
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups

In [3]:
docs = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))['data']
docs[:5]

["\n\nI am sure some bashers of Pens fans are pretty confused about the lack\nof any kind of posts about the recent Pens massacre of the Devils. Actually,\nI am  bit puzzled too and a bit relieved. However, I am going to put an end\nto non-PIttsburghers' relief with a bit of praise for the Pens. Man, they\nare killing those Devils worse than I thought. Jagr just showed you why\nhe is much better than his regular season stats. He is also a lot\nfo fun to watch in the playoffs. Bowman should let JAgr have a lot of\nfun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final\nregular season game.          PENS RULE!!!\n\n",
 'My brother is in the market for a high-performance video card that supports\nVESA local bus with 1-2MB RAM.  Does anyone have suggestions/ideas on:\n\n  - Diamond Stealth Pro Local Bus\n\n  - Orchid Farenheit 1280\n\n  - ATI Graphics Ultra Pro\n\n  - Any other high-per

In [4]:
len(docs)

18846

### 3. 토픽모델링
---------------
BERTopic 모델 객체를 만들고, fit_transform 메소드에 문자열 리스트를 입력으로 넣으면 토픽모델링을 수행함.

In [5]:
model = BERTopic()
topics, probabilities = model.fit_transform(docs)

In [7]:
print('각 문서의 토픽번호 리스트:', len(topics))
print('첫번째 문서의 토픽번호:', topics[0])

각 문서의 토픽번호 리스트: 18846
첫번째 문서의 토픽번호: 0


In [8]:
# 토픽의 개수, 토픽의 크기, 각 토픽에 할당된 단어 확인!!
model.get_topic_info()

,Topic,Count,Name
0,-1,6467,-1_to_is_the_and
1,0,1825,0_game_team_games_he
2,1,567,1_key_clipper_chip_encryption
3,2,525,2_ites_cheek_yep_huh
4,3,461,3_israel_israeli_jews_arab
...,...,...,...
212,211,10,211_needles_syringe_hypodermic_disposable
213,212,10,212_ide_bus_controller_mbs
214,213,10,213_8051_decoder_enable_demux
215,214,10,214_zoroastrians_osiris_masonry_nimrod


Count열의 값을 모두 합치면 총 문서의 수임

In [9]:
model.get_topic_info()['Count'].sum()

18846

위의 출력에서는 Topic -1이 가장 큰 것으로 보임. -1은 토픽이 할당되지 않은 모든 이상치(outliers) 문서를 나타냄. 현재 0번 토픽부터 210번 토픽까지 있는데, 임의로 5번 토픽에 대해 단어들을 출력해 확인함. get_topic() 메소드의 입력으로 보고자 하는 토픽의 번호를 넣어줌.

In [10]:
model.get_topic(5)

[('health', 0.009657636754274265),
 ('newsgroup', 0.008964955970771295),
 ('cancer', 0.00796612504048233),
 ('email', 0.007050438362084378),
 ('disease', 0.006842329283813304),
 ('tobacco', 0.006760729495320157),
 ('1993', 0.006715591150978763),
 ('vote', 0.006634122065454973),
 ('medical', 0.006579273366196812),
 ('votes', 0.006378517284922386)]

### 4. 토픽 시각화
------------
BERTopic을 사용하면 LDAvis와 매우 유사한 방식으로 생성된 토픽을 시각화할 수 있음. 

In [11]:
model.visualize_topics()

### 5. 단어 시각화
--------------


In [15]:
model.visualize_barchart()

### 6. 토픽 유사도 시각화
-------------------
각 토픽들이 서로 얼마나 유사한지 시각화 가능

In [16]:
model.visualize_heatmap()

### 7. 토픽 수 정하기
-----------------
토픽의 수를 직접 정하는 방법은 여러가지 있음. 
첫번째 방법은 모델 객체 생성시에 nr_topics 값으로 원하는 토픽수를 입력하여 원하는 토픽 수를 설정할 수 있음. BERTopic은 유사한 토픽을 찾아 하나의 토픽으로 병합함.

In [17]:
model = BERTopic(nr_topics=20)
topics, probabilities = model.fit_transform(docs)

In [18]:
model.visualize_topics()

또 다른 방법으로 모델이 자동으로 토픽 수를 줄이도록 설정하는 것임. "nr_topics" 값을 'auto'로 설정함

In [19]:
model = BERTopic(nr_topics="auto")
topics, probabilities = model.fit_transform(docs)

In [20]:
model.get_topic_info()

,Topic,Count,Name
0,-1,6833,-1_the_to_and_of
1,0,1834,0_game_team_he_games
2,1,1432,1_image_jpeg_for_file
3,2,803,2_god_that_he_of
4,3,791,3_bike_car_for_my
...,...,...,...
136,135,11,135_uv_bulbs_bulb_flashlight
137,136,11,136_8051_assembler_signetics_microcontroller
138,137,10,137_jody_news_threads_newsgroup
139,138,10,138_prophecies_lord_prophecy_god


토픽 수를 지정하지 않았을 때는 0번 토픽부터 210번 토픽까지 총 211개 토픽이 존재하였으나, 자동으로 토픽의 수가 줄어들도록 설정한 후에는 토픽 수가 0번부터 139번까지 총 140개로 줄어듦.


### 8. 임의의 문서에 대한 예측
--------------------

In [23]:
new_doc = docs[0]
print(new_doc)



I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they
are killing those Devils worse than I thought. Jagr just showed you why
he is much better than his regular season stats. He is also a lot
fo fun to watch in the playoffs. Bowman should let JAgr have a lot of
fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final
regular season game.          PENS RULE!!!




In [24]:
topics, probs = model.transform([new_doc])
print('예측한 토픽 번호:', topics)

/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



예측한 토픽 번호: [0]


### 9. 모델 저장과 로드
-------------------

In [25]:
model.save('my_topics_model')
BertTopic_model = BERTopic.load('my_topics_model')